In [17]:
# import necessary packages
import os
import cv2
import time
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Activation, Dropout

In [18]:
img_size = 100
datadir = r'C:\Users\reaz\Downloads\dataset'
CATEGORIES = os.listdir(datadir)
print(CATEGORIES)

['mask', 'no_mask']


In [19]:
# Define two empty list for contain image data
x, y = [], []
   
def PreProcess():
    for category in CATEGORIES:
        path = os.path.join(datadir, category)
        classIndex = CATEGORIES.index(category)
        print(path)
        for imgs in tqdm(os.listdir(path)):
            img_arr = cv2.imread(os.path.join(path, imgs))
            
            # resize the image
            resized_array = cv2.resize(img_arr, (img_size, img_size))
            cv2.imshow("images", resized_array)
            cv2.waitKey(1)
            resized_array = resized_array/255.0
            x.append(resized_array)
            y.append(classIndex)
            
PreProcess()
cv2.destroyAllWindows()

C:\Users\reaz\Downloads\dataset\mask


100%|██████████| 2994/2994 [00:46<00:00, 64.42it/s]


C:\Users\reaz\Downloads\dataset\no_mask


100%|██████████| 2994/2994 [00:48<00:00, 61.39it/s]


In [20]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [21]:
# Convert and resize the data to a numpy array
X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [22]:
batch_size = 32
epochs = 15

In [23]:
# Create the model architecture

model = Sequential()

model.add(Conv2D(64,(3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='relu'))


model.add(Dense(len(CATEGORIES)))
model.add(Activation('softmax'))

# compile the model

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 98, 64)        1792      
                                                                 
 activation_5 (Activation)   (None, 98, 98, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 256)       147712    
                                                                 
 activation_6 (Activation)   (None, 47, 47, 256)       0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 256)      0         
 2D)                                                  

In [25]:
t1 = time.time()

# fit the model
model.fit(X_train, y_train, batch_size = batch_size, epochs=5, validation_split=0.3, verbose = 1)
model.save('{}.h5'.format("model"))

t2 = time.time()
print('Time taken: ',t2-t1)

Epoch 1/5
105/105 [==============================] - 197s 2s/step - loss: 0.3440 - accuracy: 0.8533 - val_loss: 0.1841 - val_accuracy: 0.9562
Epoch 2/5
105/105 [==============================] - 312s 3s/step - loss: 0.1567 - accuracy: 0.9487 - val_loss: 0.1621 - val_accuracy: 0.9464
Epoch 3/5
105/105 [==============================] - 186s 2s/step - loss: 0.1369 - accuracy: 0.9547 - val_loss: 0.0993 - val_accuracy: 0.9659
Epoch 4/5
105/105 [==============================] - 182s 2s/step - loss: 0.1082 - accuracy: 0.9654 - val_loss: 0.0980 - val_accuracy: 0.9729
Epoch 5/5
105/105 [==============================] - 282s 3s/step - loss: 0.0924 - accuracy: 0.9672 - val_loss: 0.1027 - val_accuracy: 0.9659
Time taken:  1159.735873222351


In [ ]:
validation_loss, validation_accuracy = model.evaluate(X_test, y_test)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
model = load_model('model.h5')

# Define mediapipe Face detector

face_detection = mp.solutions.face_detection.FaceDetection()

# Detection function



def get_detection(frame):

    height, width, channel = frame.shape

    # Convert frame BGR to RGB colorspace

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect results from the frame
    
    result = face_detection.process(imgRGB)


    try:
        for count, detection in enumerate(result.detections):

            # print(detection)

        
            # Extract bounding box information 
            
            box = detection.location_data.relative_bounding_box

            x, y, w, h = int(box.xmin*width), int(box.ymin * height), int(box.width*width), int(box.height*height)
            
    # If detection is not available then pass 
    except:
        pass

    return x, y, w, h

CATEGORIES = ['no_mask', 'mask']
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    img = frame.copy()
    try:
        x, y, w, h = get_detection(frame)
        
        crop_img = img[y:y+h, x:x+w]
        
        crop_img = cv2.resize(crop_img, (100, 100))
        
        crop_img = np.expand_dims(crop_img, axis=0)
        
        # get the prediction from the model.
        prediction = model.predict(crop_img)
        print(prediction)
        index = np.argmax(prediction)
        res = CATEGORIES[index]
        if index == 0:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, res, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                                 0.8, color, 2, cv2.LINE_AA)

    except:
        pass
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 630ms/step
[[0. 1.]]
1/1 [==============================] - 0s 88ms/step
[[0. 1.]]
1/1 [==============================] - 0s 77ms/step
[[0. 1.]]
1/1 [==============================] - 0s 91ms/step
[[0. 1.]]
1/1 [==============================] - 0s 73ms/step
[[0. 1.]]
1/1 [==============================] - 0s 86ms/step
[[0. 1.]]
1/1 [==============================] - 0s 88ms/step
[[0. 1.]]
1/1 [==============================] - 0s 134ms/step
[[0. 1.]]
1/1 [==============================] - 0s 94ms/step
[[0. 1.]]
1/1 [==============================] - 0s 85ms/step
[[0. 1.]]
1/1 [==============================] - 0s 81ms/step
[[0. 1.]]
1/1 [==============================] - 0s 81ms/step
[[0. 1.]]
1/1 [==============================] - 0s 77ms/step
[[0. 1.]]
1/1 [==============================] - 0s 72ms/step
[[0. 1.]]
1/1 [==============================] - 0s 70ms/step
[[0. 1.]]
1/1 [==============================] - 0s 76ms/step
[[0. 1.]]
1/1 [=